<a href="https://colab.research.google.com/github/andronikmk/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/ANDRONIK_MKRTYCHEV_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [X] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import numpy as np
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature
# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [167]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019


### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.###

In [168]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].describe()

count                   23040
unique                    120
top       2019-01-24 00:00:00
freq                      480
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [169]:
df['SALE_DATE'].dt.month.value_counts()

1    6464
3    6105
2    5598
4    4873
Name: SALE_DATE, dtype: int64

In [0]:
train = df[(df.SALE_DATE.dt.month >= 1) & (df.SALE_DATE.dt.month <= 3)]

In [0]:
test = df[df.SALE_DATE.dt.month == 4]

In [0]:
train = train.drop(columns='EASE-MENT')

In [0]:
test = test.drop(columns='EASE-MENT')

In [0]:
train = train.dropna()
test = test.dropna()

In [175]:
train.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
APARTMENT_NUMBER                  0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [176]:
test.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
APARTMENT_NUMBER                  0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [177]:
train.shape, test.shape

((3940, 20), (1176, 20))

###Do one-hot encoding of categorical features.###

In [178]:
# column numerics (exploring data)
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,3940.0,2.487269e+03,2.540336e+03,1.0,878.0,1446.0,3402.75,16234.0
LOT,3940.0,1.262651e+03,6.274599e+02,1.0,1032.0,1117.0,1305.00,9022.0
ZIP_CODE,3940.0,1.063671e+04,6.471155e+02,0.0,10019.0,10462.0,11222.00,11694.0
RESIDENTIAL_UNITS,3940.0,1.493401e+00,8.702333e+00,0.0,1.0,1.0,1.00,428.0
COMMERCIAL_UNITS,3940.0,6.137056e-01,8.323585e+00,-1.0,0.0,0.0,0.00,313.0
TOTAL_UNITS,3940.0,2.806853e+00,1.383706e+01,0.0,1.0,1.0,1.00,428.0
GROSS_SQUARE_FEET,3940.0,3.554246e+03,1.649318e+04,0.0,649.0,949.0,1432.25,349126.0
YEAR_BUILT,3940.0,1.590241e+03,7.902565e+02,0.0,1913.0,1985.0,2009.00,2018.0
TAX_CLASS_AT_TIME_OF_SALE,3940.0,2.188325e+00,7.374684e-01,1.0,2.0,2.0,2.00,4.0
SALE_PRICE,3940.0,1.654465e+06,7.255767e+06,0.0,0.0,615000.0,1325000.00,239958219.0


In [179]:
# column numberics exclude numbers
# What are the catagories with high cardinality?
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BOROUGH,3940,5,1,1797,NaT,NaT
TAX_CLASS_AT_PRESENT,3940,8,2,2741,NaT,NaT
NEIGHBORHOOD,3940,11,OTHER,2852,NaT,NaT
BUILDING_CLASS_CATEGORY,3940,22,13 CONDOS - ELEVATOR APARTMENTS,2529,NaT,NaT
BUILDING_CLASS_AT_PRESENT,3940,36,R4,2529,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,3940,36,R4,2529,NaT,NaT
SALE_DATE,3940,83,2019-01-24 00:00:00,219,2019-01-01,2019-03-31
LAND_SQUARE_FEET,3940,618,0,2295,NaT,NaT
APARTMENT_NUMBER,3940,1448,4,81,NaT,NaT
ADDRESS,3940,3887,"1335 AVENUE OF THE AMERICAS, HU2",3,NaT,NaT


In [180]:
# Relationship Borough and Price
train['BOROUGH'].value_counts()

1    1797
3    1241
4     656
2     151
5      95
Name: BOROUGH, dtype: int64

In [181]:
train.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    2.591269e+06
2    2.793887e+05
3    8.212007e+05
4    1.184977e+06
5    2.467353e+05
Name: SALE_PRICE, dtype: float64

In [182]:
train.groupby('BOROUGH')['SALE_PRICE'].describe()

,count,mean,std,min,25%,50%,75%,max
BOROUGH,,,,,,,,
1,1797.0,2.591269e+06,9.796041e+06,0.0,0.0,950000.0,2316949.0,239958219.0
2,151.0,2.793887e+05,9.345211e+05,0.0,69250.0,175000.0,287500.0,11500000.0
3,1241.0,8.212007e+05,1.457413e+06,0.0,10.0,629279.0,998000.0,28028095.0
4,656.0,1.184977e+06,6.676916e+06,0.0,30315.0,472500.0,748810.5,75000000.0
5,95.0,2.467353e+05,1.493933e+05,0.0,207449.0,260000.0,350000.0,610849.0


In [0]:
# Data you want to drop includes target and high cardinality
target = 'SALE_PRICE'
high_cardinality = ['SALE_DATE','BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE','APARTMENT_NUMBER','LAND_SQUARE_FEET','ADDRESS','YEAR_BUILT']
features = train.columns.drop([target] + high_cardinality)

In [0]:
# Drop df columns into test and train
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [185]:
X_train.shape

(3940, 12)

In [186]:
X_test.shape

(1176, 12)

In [0]:
# TODO
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [0]:
X_test = encoder.transform(X_test)

In [189]:
X_train.shape

(3940, 54)

In [190]:
X_test.shape

(1176, 54)

## Do feature selection with SelectKBest##

In [191]:
X_train.shape, X_test.shape

((3940, 54), (1176, 54))

In [193]:
# TODO: Select the 15 features that best correlate with the target
# (15 is an arbitrary starting point here)
from sklearn.feature_selection import SelectKBest, f_regression

# SelectKBest has a similar API to what we've seen before.
selector = SelectKBest(score_func=f_regression, k=15)

# IMPORTANT!
# .fit_transform on the train set
# .transform on test set
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((3940, 15), (1176, 15))

In [194]:
# TODO: Which features were selected?
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\nFeatures not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_1
BOROUGH_3
NEIGHBORHOOD_UPPER EAST SIDE (59-79)
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_45 CONDO HOTELS
BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_11A CONDO-RENTALS
BUILDING_CLASS_CATEGORY_28 COMMERCIAL CONDOS
BLOCK
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
TAX_CLASS_AT_TIME_OF_SALE

Features not selected:
BOROUGH_4
BOROUGH_5
BOROUGH_2
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_UPPER EAST SIDE (79-96)
NEIGHBORHOOD_UPPER WEST SIDE (59-79)
NEIGHBORHOOD_UPPER WEST SIDE (79-96)
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_GRAMERCY
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FOREST HILLS
BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL
BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS
BUILDING_CLASS_CATEGORY_44 CONDO PARKING
BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE
BUILDING_CLASS_CATEGORY_12 COND

In [210]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $2,644,945 

2 features
Test Mean Absolute Error: $2,650,697 

3 features
Test Mean Absolute Error: $2,615,257 

4 features
Test Mean Absolute Error: $2,686,791 

5 features
Test Mean Absolute Error: $2,612,083 

6 features
Test Mean Absolute Error: $2,608,664 

7 features
Test Mean Absolute Error: $2,607,562 

8 features
Test Mean Absolute Error: $2,668,888 

9 features
Test Mean Absolute Error: $2,682,128 

10 features
Test Mean Absolute Error: $2,694,247 

11 features
Test Mean Absolute Error: $2,677,164 

12 features
Test Mean Absolute Error: $2,685,935 

13 features
Test Mean Absolute Error: $2,685,013 

14 features
Test Mean Absolute Error: $2,686,847 

15 features
Test Mean Absolute Error: $2,704,829 

16 features
Test Mean Absolute Error: $2,706,277 

17 features
Test Mean Absolute Error: $2,706,279 

18 features
Test Mean Absolute Error: $2,705,775 

19 features
Test Mean Absolute Error: $2,707,073 

20 features
Test Mean Absolute Error: $2